In [1]:
import csv
import json
import pandas as pd
import sys, getopt, pprint
from pymongo import MongoClient
from pandas import DataFrame


#df=pd.read_csv('/opt/fichiers/projet/version2_0.csv', delimiter=";")
data = DataFrame(df, columns = ["nom","prenom","adresse","ville","region","nb_identite"])
dfj=df.to_json('Documents/projet/yourjson.json',orient='table')
jdf = open('Documents/projet/yourjson.json').read()
data = json.loads(jdf)

client = MongoClient('localhost',27017)
db = client.db
collection = db.collection


db.collection.insert_one(data)

NameError: name 'df' is not defined

In [39]:
from pyspark.sql import DataFrameReader, DataFrameReader
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark

spark = SparkSession.builder.appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/malade.contamine?readPreference=primaryPreferred") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/malade.contamine") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0') \
    .getOrCreate()

df_csv = spark.read.csv("/opt/fichiers/projet/version2_0.csv", header=True,mode="DROPMALFORMED",sep=";")
df_csv.write.format("com.mongodb.spark.sql.DefaultSource").mode('append').option('database',"malade").option('collection',"contamine").save()
df01 = spark.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","malade")\
    .option("collection", "contamine")\
    .load()

df01.show()

+--------------------+--------------------+-----------+---------+--------+-------------+------------+
|                 _id|             adresse|nb_identite|      nom|  prenom|       region|       ville|
+--------------------+--------------------+-----------+---------+--------+-------------+------------+
|[5ecd03454a71cb0c...|    East Main Street|    1437018|  Johnson| Millard|      Vermont|       Dover|
|[5ecd03454a71cb0c...|       Richmond Hill|     119595|  Harding|  Gerald|      Florida| Little Rock|
|[5ecd03454a71cb0c...|      Cleveland Ave.|    2303298|  Johnson|  Gerald|       Kansas|  Harrisburg|
|[5ecd03454a71cb0c...|   San Diego Freeway|    8070934|  Clinton|  Thomas|     Missouri|   Frankfort|
|[5ecd03454a71cb0c...|        Camelback Rd|    3255890|    Hayes|  Thomas|   New Jersey|      Boston|
|[5ecd03454a71cb0c...|    East Main Street|    1549796| Harrison|  Grover|        Maine|     Atlanta|
|[5ecd03454a71cb0c...|      Cleveland Ave.|    3875962| Harrison|  Warren|      Ve

In [18]:
from pyspark.sql import *
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark
from pyspark.sql.functions import *
import random



# first connector to connect spark and mongo
spark_malade = SparkSession.builder.appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/population.coronavirus?readPreference=primaryPreferred") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/population.coronavirus") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0') \
    .getOrCreate()

# second connector to connect spark and mongo
spark_contamine = SparkSession.builder.appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/population.contamine?readPreference=primaryPreferred") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/population.contamine") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0') \
    .getOrCreate()

# third connector to connect spark and mongo to make SQL request
spark = SparkSession.builder \
    .master("local") \
    .appName("myApp") \
    .enableHiveSupport() \
    .getOrCreate()


#spark read csv file
df_csv_malade = spark_malade.read.csv("/opt/fichiers/projet/malade.csv", header=True,mode="DROPMALFORMED",sep=";")

#spark send dataframe to mongodb
df_csv_malade.write.format("com.mongodb.spark.sql.DefaultSource").mode('overwrite').option('database',"population").option('collection',"coronavirus").save()

#spark read second csv file
df_csv_contamine = spark_contamine.read.csv("/opt/fichiers/projet/contamine.csv", header=True,mode="DROPMALFORMED",sep=";")

#spark send dataframe to mongodb
df_csv_contamine.write.format("com.mongodb.spark.sql.DefaultSource").mode('overwrite').option('database',"population").option('collection',"contamine").save()


df01 = spark_malade.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","population")\
    .option("collection", "coronavirus")\
    .load()

df02 = spark_contamine.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","population")\
    .option("collection", "contamine")\
    .load()

#cross 2 dfs
df_final = df02.crossJoin(df01)

#list of tuple and convert to rdd
date_rencontre =[(9,1,1,"21/02",3),(9,2,2,"22/02",2),(9,3,3,"21/02",1),(9,4,4,"22/02",0),(9,5,5,"21/02",0),(9,6,6,"22/02",1),(9,7,7,"21/02",2),(9,8,8,"22/02",3),(10,1,9,"21/02",3),(10,2,10,"22/02",2),(10,3,11,"21/02",1),(10,4,12,"22/02",0),(10,5,13,"21/02",1),(10,6,14,"22/02",1),(10,7,15,"21/02",2),(10,8,16,"22/02",3)]
rdd_date = spark.sparkContext.parallelize(date_rencontre)

#rdd to map to df
map_date = rdd_date.map(lambda x:Row(Id_contamine_C=x[0],Id_personne =x[1],id_date=x[2],date_rencontre=x[3],distance=x[4]))
df_date = spark.createDataFrame(map_date)
   
#create 2 temps views to do sql request
df_final.createOrReplaceTempView("sortie")
df_date.createOrReplaceTempView("date")

#sql request
request_sec = spark.sql("SELECT sortie.*,date_rencontre,distance FROM sortie inner join date on sortie.Id_personne = date.Id_personne and sortie.Id_contamine_C = date.Id_contamine_C Where distance <= 1")

#show me result
request_sec.show()


+--------------------+--------------+----------+--------+------------+--------------+--------------------+-------------+---------------+-----------+---------+--------+------------+-----------+--------------------+-------------+--------------+--------+
|           Adresse_C|Id_contamine_C|     Nom_C|Prenom_C|    Region_C|       Ville_C|                 _id|nb_identite_C|      Adresse_P|Id_personne|    Nom_P|Prenom_P|    Region_P|    Ville_P|                 _id|nb_identite_P|date_rencontre|distance|
+--------------------+--------------+----------+--------+------------+--------------+--------------------+-------------+---------------+-----------+---------+--------+------------+-----------+--------------------+-------------+--------------+--------+
|San Luis Obispo N...|             9|Eisenhower|  Ronald|South Dakota|Salt Lake City|[5edeea534a71cb23...|       214334|E Fowler Avenue|          6|Jefferson| Zachary|Rhode Island|Carson City|[5edeea534a71cb23...|        31775|         22/02|  